<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GEMINI3_VJEPA_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install av -q

In [2]:
# Cell 1: Conceptual Modifications - Aviation Data Definitions

import torch
import numpy as np
import os
import glob
import av
import json
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoVideoProcessor, AutoModel
from tqdm.auto import tqdm
import logging
import datetime
import pytz
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s')


class AgentConfig:
    # --- REFLECTS NEW GEMINI MODEL ID ---
    LLM_MODEL_NAME: str = "gemini-3-pro-preview"
    CLASS_LABELS = [
        "airplane landing",
        "airplane takeoff",
        "airport ground operations",
        "in-flight cruise",
        "emergency landing",
        "pre-flight check/maintenance",
        "en-route cruise",
        "climb phase",
        "descent phase",
        "holding pattern"
    ]

# Define num_classes globally
num_classes = len(AgentConfig.CLASS_LABELS)

# --- FIX: CLASSIFIER_SAVE_PATH moved to global scope ---
CLASSIFIER_SAVE_PATH = "classifier_head_trained_on_tartan_aviation_sample.pth"

AIRPORTS = {
    "CYUL": {"name": "Montreal-Trudeau International", "lat": 45.4706, "lon": -73.7408, "elevation_ft": 118},
    "LFPG": {"name": "Paris-Charles de Gaulle", "lat": 49.0097, "lon": 2.5479, "elevation_ft": 392},
}

AIRCRAFT_PERFORMANCE = {
    "Boeing777_300ER": {
        "cruise_speed_knots": 490,
        "fuel_burn_kg_per_hour": 7000,
        "max_range_nm": 7900,
        "climb_rate_fpm": 2500,
        "descent_rate_fpm": 2000,
        "typical_cruise_altitude_ft": 37000,
        "fuel_capacity_kg": 145000
    }
}

hf_repo = "facebook/vjepa2-vitg-fpc64-256"
EXTRACTED_FEATURES_DIR = "/content/gdrive/MyDrive/datasets/TartanAviation_VJEPA_Features/"

TOTAL_FLATTENED_VJEPA_DIM = 2048 * 1408

CONCEPTUAL_PLDM_LATENT_DIM = 1024

latent_dim_pldm = CONCEPTUAL_PLDM_LATENT_DIM
action_dim = 8

def load_and_process_video(video_path, processor_instance, model_instance, device_instance, num_frames_to_sample=16):
    """
    Loads a video, samples frames, and extracts V-JEPA features.
    Returns extracted features (torch.Tensor, shape like [1, 2048, 1408]) and the frames.
    Does NOT flatten the V-JEPA output here, keeping it as model's raw output.
    """
    frames = []
    if not os.path.exists(video_path):
        logging.error(f"ERROR: Video file '{video_path}' not found.")
        return None, None
    try:
        container = av.open(video_path)
        total_frames_in_video = container.streams.video[0].frames
        sampling_interval = max(1, total_frames_in_video // num_frames_to_sample)
        logging.info(f"Total frames in video: {total_frames_in_video}")
        logging.info(f"Sampling interval: {sampling_interval} frames")

        for i, frame in enumerate(container.decode(video=0)):
            if len(frames) >= num_frames_to_sample:
                break
            if i % sampling_interval == 0:
                img = frame.to_rgb().to_ndarray()
                frames.append(img)

        if not frames:
            logging.error(f"ERROR: No frames could be loaded from '{video_path}'.")
            return None, None
        elif len(frames) < num_frames_to_sample:
            logging.warning(f"WARNING: Only {len(frames)} frames loaded. Requested: {num_frames_to_sample}.")

        inputs = processor_instance(videos=list(frames), return_tensors="pt")
        inputs = {k: v.to(device_instance) for k, v in inputs.items()}

        with torch.no_grad():
            features = model_instance(**inputs).last_hidden_state

        logging.info(f"Successfully extracted V-JEPA features with raw shape: {features.shape}")
        return features, frames

    except av.FFmpegError as e:
        logging.error(f"Error loading video with PyAV: {e}")
        logging.error("This might indicate an issue with the video file itself or PyAV installation.")
        return None, None
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        logging.error("Ensure 'av' library is installed (`pip install av`) and video file is not corrupt.")
        return None, None

class ClassifierHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        return self.fc2(self.dropout(self.relu(self.fc1(x))))

class LatentDynamicsPredictor(torch.nn.Module):
    def __init__(self, latent_dim, action_dim):
        super().__init__()
        self.layers = torch.nn.Sequential(
            nn.Linear(latent_dim + action_dim, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )

    def forward(self, latent_state, action):
        combined_input = torch.cat([latent_state, action], dim=-1)
        predicted_next_latent_state = self.layers(combined_input)
        return predicted_next_latent_state

class LatentProjector_old(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.projector = nn.Linear(input_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.projector(x))


class LatentProjector(nn.Module):
    def __init__(self, input_dim=4, output_dim=1024):
        super().__init__()
        self.projector = nn.Linear(input_dim, output_dim)
        self.relu = nn.ReLU()


print("\n--- Instantiating Models and Optimizers ---")
model = AutoModel.from_pretrained(hf_repo)
processor = AutoVideoProcessor.from_pretrained(hf_repo)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

latent_projector = LatentProjector(TOTAL_FLATTENED_VJEPA_DIM, CONCEPTUAL_PLDM_LATENT_DIM)
latent_projector.to(device)

predictor = LatentDynamicsPredictor(latent_dim_pldm, action_dim)
predictor.to(device)
optimizer_pldm = torch.optim.Adam(list(predictor.parameters()) + list(latent_projector.parameters()), lr=0.001)

classifier = ClassifierHead(input_dim=1408, num_classes=num_classes)
classifier.to(device)

print(f"Models instantiated and moved to {device}.")
print("Cell 1 setup complete for conceptual flight planning.")


--- Instantiating Models and Optimizers ---
Models instantiated and moved to cuda.
Cell 1 setup complete for conceptual flight planning.


In [3]:
from google.colab import userdata
import google.generativeai as genai
from google.generativeai import types

# --- Setup ---
# 1. Retrieve the API key securely from Colab's user data
# NOTE: The secret name is assumed to be 'GEMINI' in Colab Secrets
GOOGLE_API_KEY = userdata.get('GEMINI')

if not GOOGLE_API_KEY:
    print("Error: 'GEMINI' API key not found in Colab Secrets.")
    # You may choose to exit here if the key is mandatory

try:
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"Error configuring Gemini API: {e}")

# 2. Change the model identifier to target Gemini 3 Pro Preview
MODEL_ID = 'gemini-3-pro-preview'

# 3. Initialize the GenerativeModel object with the correct ID
try:
    # The global model object for generate_content()
    gemini_model = genai.GenerativeModel(MODEL_ID)
    print(f"Gemini client successfully configured for model: {MODEL_ID}.")
except Exception as e:
    print(f"Error initializing Gemini Model: {e}")

# Update AgentConfig (although it's already set in Cell 1, this confirms it)
# Assuming AgentConfig is available globally from Cell 1.
try:
    AgentConfig.LLM_MODEL_NAME = MODEL_ID
except NameError:
    print("WARNING: AgentConfig not yet defined. Proceeding with global variable setting.")
    # For safety if Cell 1 wasn't run first:
    pass

Gemini client successfully configured for model: gemini-3-pro-preview.


In [ ]:
#Cell 2: Core Execution Feature Extraction, Classifier Training & Inference, LLM Interaction, and PLDM Training/Planning
# This cell assumes Cell 1 has been successfully executed in the current session.
import os
import logging
import torch
import json
from google.colab import drive
from tqdm.auto import tqdm
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import datetime
import pytz
# Ensure genai is imported if you run this cell independently
import google.generativeai as genai
from google.generativeai import types

#Mounting Google Drive
print("\n--- Cell 2: Mounting Google Drive for dataset access ---")
drive.mount('/content/gdrive')
print("Google Drive mounted.")

print(f"Checking for extracted features directory: {EXTRACTED_FEATURES_DIR}")
if not os.path.exists(EXTRACTED_FEATURES_DIR):
    logging.error(f"ERROR: Extracted features directory '{EXTRACTED_FEATURES_DIR}' not found. Please create it and upload V-JEPA features.")
    # Note: In a real notebook, you would continue the flow. Using 'exit()' is aggressive.
    # For a demo, we assume the directory will be created or is not strictly required for the classifier part.
else:
    print(f"Extracted features directory found at {EXTRACTED_FEATURES_DIR}")

# Part 1: Load and process airplane-landing.mp4 for initial observation
print(f"\n--- Cell 2: Part 1 - Loading actual video '/content/gdrive/MyDrive/datasets/TartanAviation_VJEPA_Features/airplane-landing.mp4' for feature extraction ---")
flight_video_path = '/content/gdrive/MyDrive/datasets/TartanAviation_VJEPA_Features/airplane-landing.mp4'
video_features_for_inference_raw, frames_for_pldm_planning = load_and_process_video(flight_video_path, processor, model, device_instance=device)

# -- CRITICAL: Process raw V-JEPA features to match ClassifierHead's expected input_dim --
if video_features_for_inference_raw is not None:
    pooled_features_for_classifier = video_features_for_inference_raw.squeeze(0).mean(dim=0).unsqueeze(0)
    extracted_embedding_dim_for_classifier = pooled_features_for_classifier.shape[-1]
    logging.info(f"Dynamically determined extracted_embedding_dim for ClassifierHead: {extracted_embedding_dim_for_classifier}")
else:
    pooled_features_for_classifier = None
    extracted_embedding_dim_for_classifier = -1
    logging.error("Failed to extract video features for classifier. Skipping inference part.")
    # Set a flag to skip later parts that depend on this
    skip_inference = True


# Part 2: Classifier Training (Code remains the same as original)
print(f"\n--- Cell 2: Part 2 - Starting Classifier Training ---")
try:
    # Re-initialize classifier with the correct, dynamically determined input_dim
    if extracted_embedding_dim_for_classifier != -1:
        classifier = ClassifierHead(input_dim=extracted_embedding_dim_for_classifier, num_classes=num_classes)
        classifier.to(device)

        # ... (Data loading and training logic from original Cell 2) ...
        train_features_list = []
        train_labels_list = []

        map_file_path = os.path.join(EXTRACTED_FEATURES_DIR, "feature_label_map.json")
        if not os.path.exists(map_file_path):
            logging.warning(f"Feature-label map file '{map_file_path}' not found. Generating synthetic data.")
            feature_label_map = {}
        else:
            with open(map_file_path, 'r') as f:
                feature_label_map = json.load(f)

        if not feature_label_map:
            logging.warning(f"Feature-label map at {map_file_path} is empty. Generating synthetic data.")
            num_training_samples = 2_000_000
            # Synthetic data generation uses the dynamically determined input_dim
            train_features = torch.rand(num_training_samples, extracted_embedding_dim_for_classifier)
            train_labels = torch.randint(0, num_classes, (num_training_samples,))
            train_loader = DataLoader(TensorDataset(train_features, train_labels), batch_size=32, shuffle=True)
            val_loader = None
            print(f"Loaded {num_training_samples} SYNTHETIC features for training.")
        else:
            for item in tqdm(feature_label_map, desc="Loading real V-JEPA features"):
                feature_path = item['feature_path']
                label_idx = item['label_idx']
                try:
                    if not os.path.isabs(feature_path):
                        feature_path = os.path.join(EXTRACTED_FEATURES_DIR, feature_path)

                    if not os.path.exists(feature_path):
                        logging.warning(f"Feature file not found at {feature_path}. Skipping.")
                        continue

                    feature = torch.load(feature_path, map_location=device)

                    # Match your working code's pooling/squashing logic to get [1408] dim
                    if feature.ndim == 3:
                        feature = feature.squeeze(0).mean(dim=0)
                    elif feature.ndim == 2:
                        if feature.shape[0] == 1 and feature.shape[1] == 1408:
                            feature = feature.squeeze(0)
                        elif feature.shape[1] == 1408:
                            feature = feature.mean(dim=0)
                        else:
                            feature = feature.flatten()
                    elif feature.ndim == 1:
                        pass
                    else:
                        logging.warning(f"Skipping malformed feature from {feature_path}. Unexpected dimensions: {feature.ndim}")
                        continue

                    # Final check after processing. Should be 1D with 1408 elements.
                    if feature.shape[0] != extracted_embedding_dim_for_classifier:
                        logging.warning(f"Skipping feature at {feature_path}. Dimension mismatch: expected {extracted_embedding_dim_for_classifier}, got {feature.shape[0]}.")
                        continue

                    train_features_list.append(feature)
                    train_labels_list.append(label_idx)

                except Exception as e:
                    logging.error(f"Error loading feature from {feature_path}: {e}. Skipping.")

            if train_features_list:
                train_features = torch.stack(train_features_list).to(device)
                train_labels = torch.tensor(train_labels_list).to(device)
                num_training_samples = len(train_features)
                print(f"Loaded {num_training_samples} REAL V-JEPA features for training.")

                if num_training_samples < 2:
                    print("WARNING: Only 1 real V-JEPA feature loaded. Training may be unstable. Consider more data.")
                    train_loader = DataLoader(TensorDataset(train_features, train_labels), batch_size=1, shuffle=True)
                    val_loader = None
                else:
                    dataset_size = len(train_features)
                    train_size = int(0.8 * dataset_size)
                    val_size = dataset_size - train_size

                    if val_size == 0 and train_size > 0:
                        train_size = dataset_size
                        train_dataset_real = TensorDataset(train_features, train_labels)
                        val_dataset_real = None
                    else:
                        train_dataset_real, val_dataset_real = torch.utils.data.random_split(
                            TensorDataset(train_features, train_labels), [train_size, val_size]
                        )
                    train_loader = DataLoader(train_dataset_real, batch_size=32, shuffle=True)
                    val_loader = DataLoader(val_dataset_real, batch_size=32, shuffle=False) if val_dataset_real else None
                    print(f"Training on {len(train_dataset_real)} samples, Validation on {len(val_dataset_real) if val_dataset_real else 0} samples.")
            else:
                logging.error("No real V-JEPA features could be loaded from map file. Generating synthetic data as fallback.")
                num_training_samples = 2_000_000
                train_features = torch.rand(num_training_samples, extracted_embedding_dim_for_classifier)
                train_labels = torch.randint(0, num_classes, (num_training_samples,))
                train_loader = DataLoader(TensorDataset(train_features, train_labels), batch_size=32, shuffle=True)
                val_loader = None
                print(f"Loaded {num_training_samples} SYNTHETIC features for training as fallback.")


        criterion = torch.nn.CrossEntropyLoss()
        optimizer_classifier = torch.optim.Adam(classifier.parameters(), lr=0.001)

        num_epochs = 20
        for epoch in range(num_epochs):
            classifier.train()
            running_loss = 0.0
            for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer_classifier.zero_grad()
                outputs = classifier(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer_classifier.step()
                running_loss += loss.item()

            epoch_loss = running_loss / len(train_loader.dataset)

            val_loss = 0.0
            if val_loader and len(val_loader.dataset) > 0:
                classifier.eval()
                with torch.no_grad():
                    for inputs, labels in val_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = classifier(inputs)
                        loss = criterion(outputs, labels)
                        val_loss += loss.item()
                val_loss /= len(val_loader.dataset)
                logging.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")
            else:
                logging.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}")
                print("No validation data available or validation dataset is empty.")


        print("\n--- Classifier Training Complete ---")
        torch.save(classifier.state_dict(), CLASSIFIER_SAVE_PATH)
        print(f"Classifier saved to: {CLASSIFIER_SAVE_PATH}")
    else:
        print("Skipping classifier training due to feature extraction failure.")

except Exception as e:
    logging.error(f"Error during classifier training: {e}")

# --- LLM Interaction (Part 3) is moved to a separate cell for clarity and dependency ---
print("Cell 2 execution complete.")

In [5]:
print(f"Classifier saved to: {CLASSIFIER_SAVE_PATH}")
print("Cell 2 execution complete.")

Classifier saved to: classifier_head_trained_on_tartan_aviation_sample.pth
Cell 2 execution complete.


In [6]:
# --- REFACTORED LLM INTERACTION (Part 3) ---

# This cell assumes the GEMINI API setup and model training (Cell 1 & Cell 2 parts 1/2) have run.

import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import numpy as np
import logging
import pytz
import datetime
# Imports required for the execution
import google.generativeai as genai
from google.generativeai import types


print("\n--- Starting V-JEPA Feature-Driven Classification Inference and GEMINI LLM Interaction ---")

# Check if pooled_features_for_classifier is available
if 'pooled_features_for_classifier' not in globals() or pooled_features_for_classifier is None:
    logging.error("ERROR: Cannot perform LLM interaction as 'pooled_features_for_classifier' is missing or None.")
else:
    try:
        pooled_features_for_inference_on_device = pooled_features_for_classifier.to(device)

        # Assuming classifier is defined and CLASSIFIER_SAVE_PATH is accessible
        try:
            classifier.load_state_dict(torch.load(CLASSIFIER_SAVE_PATH, map_location=device))
            logging.info(f"Classifier weights loaded from: {CLASSIFIER_SAVE_PATH}")
        except NameError:
             logging.error("Classifier model not found in global scope. Re-initialize or check execution flow.")
             raise

        classifier.eval()
        with torch.no_grad():
            logits = classifier(pooled_features_for_inference_on_device)
            probabilities = torch.softmax(logits, dim=1)

        predicted_class_idx = torch.argmax(probabilities, dim=1).item()
        predicted_confidence = probabilities[0, predicted_class_idx].item()
        predicted_label = AgentConfig.CLASS_LABELS[predicted_class_idx]

        # --- LLM Input Description Logic (Simplified for brevity) ---
        llm_input_description = f"The visual system detected an airplane {predicted_label}. (Confidence: {predicted_confidence:.2f})"

        print(f"\n--- AI Agent's Understanding from Classifier ---")
        print(f"**Primary Classification (Predicted by AI):** '{predicted_label}' (Confidence: {predicted_confidence:.2f})")
        print(f"**Description for LLM:** {llm_input_description}")

        # --- Engaging GEMINI LLM for Further Reasoning ---
        print(f"\n--- Engaging GEMINI LLM ({AgentConfig.LLM_MODEL_NAME}) for Further Reasoning ---")
        try:
            if 'gemini_model' not in globals():
                raise NameError("Gemini model object 'gemini_model' is not defined.")

            prompt_for_gemini = f"""
                  You are an AI assistant for flight planning operations.
                  Current visual observation: {llm_input_description}
                  Current time (EST): {datetime.datetime.now(pytz.timezone('EST')).strftime('%Y-%m-%d %H:%M:%S EST')}

                  Based on this visual observation, provide a concise operational assessment relevant to flight planning.
                  If the observation seems random or uncertain, state that. Do not add any conversational filler.
                  """

            # --- GEMINI API CALL STRUCTURE (FINAL WORKING FIX) ---
            # FIX: Only pass the 'contents' argument. This is the oldest/most minimal syntax.
            gemini_response = gemini_model.generate_content(
                contents=prompt_for_gemini
            )

            print("\n--- GEMINI LLM Response ---")
            if gemini_response.text:
                print(gemini_response.text)
                print("--- GEMINI LLM Response - END ---")
                print('\n')
            else:
                print("GEMINI LLM did not provide a text response or cannot provide one.")

        except NameError as name_e:
            logging.error(f"Error: {name_e}. Please run the Gemini API Setup cell.")
        except Exception as llm_e:
            logging.error(f"Error interacting with GEMINI LLM: {llm_e}")
            logging.error("Ensure your 'GEMINI' API key is correctly set in Colab Secrets and the model ID is valid.")

    except Exception as e:
        logging.error(f"Error during classification inference: {e}")

print(f"The V-JEPA features (shape: {pooled_features_for_classifier.shape if pooled_features_for_classifier is not None else 'N/A'}) are the core input that a trained classifier would learn from.")
print("\nLLM Interaction cell execution complete.")


--- Starting V-JEPA Feature-Driven Classification Inference and GEMINI LLM Interaction ---

--- AI Agent's Understanding from Classifier ---
**Primary Classification (Predicted by AI):** 'airplane landing' (Confidence: 1.00)
**Description for LLM:** The visual system detected an airplane airplane landing. (Confidence: 1.00)

--- Engaging GEMINI LLM (gemini-3-pro-preview) for Further Reasoning ---

--- GEMINI LLM Response ---
Confirmed aircraft landing detected. Update flight status to Actual Time of Arrival (ATA). Runway occupancy is active; prepare for taxiing, gate allocation, and turnaround servicing.
--- GEMINI LLM Response - END ---


The V-JEPA features (shape: torch.Size([1, 1408])) are the core input that a trained classifier would learn from.

LLM Interaction cell execution complete.
